In [1]:
import re
import datetime
import numpy as np
import pandas as pd 
import os
import sys
import itertools
import io

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.utils import *

### Establishes General Path

In [3]:
##local_path = "/your/path/here"
local_path = "/users/thudson/Documents/Dropbox/chicago-police-data/02-column_std"

In [4]:
path = '/Data/chicago-police-data/import'
db = dropbox_handler()
db.list_files(path+'/output')

['complaints-cpd-2016-dec',
 'complaints-cpd-2016-jun',
 'complaints-cpd-2016-nov',
 'complaints-cpd-2016-oct',
 'complaints-ipra-2016-apr',
 'complaints-merged-2015',
 'incomplete',
 'shootings-cpd-feb2016',
 'shootings-ipra-may2016',
 'TRRs']

In [5]:
in_path = path + '/output/shootings-cpd-feb2016/'
out_path = local_path + '/output/shootings-cpd-feb2016/'

In [6]:
try:
    os.mkdir(out_path)
except:
    print("path exists: {}".format(out_path))

path exists: /users/thudson/Documents/Dropbox/chicago-police-data/02-column_std/output/shootings-cpd-feb2016/


In [7]:
##doc_path = '/Data/chicago-police-data/02-column_std'
##column_dict = db.download_file(doc_path+'/doc/','Column_Dictionary.csv')

doc_path = local_path
column_dict = pd.read_csv(doc_path+'/doc/'+'Column_Dictionary.csv').iloc[:,0:3]
column_dict.head()

,Orig Column Name,New Name
0,Accused,Officer.Accused.Full_Name
1,Accused_Appointed_Date,Officer.Accused.Appointed_Date
2,Accused_Appointment_Date,Officer.Accused.Appointed_Date
3,Accused_Birth_Year,Officer.Accused.Birth_Year
4,Accused_Detail,Officer.Accused.Detail


In [8]:
## SOMETHING TO PUT IN UTILS? HOW WOULD THE SPECIFIED COLUMN_DICT WORK?

def column_mapper(df,
                  column_dict = column_dict):
    current_columns = pd.DataFrame(df.columns.tolist(),columns=['Orig Column Name'])
    new_columns = current_columns.merge(column_dict,how='left',on='Orig Column Name')['New Name'].tolist() 
    if current_columns.shape[0] == len(new_columns):
        return new_columns
    else:
        print("Join failed, more columns returned than expected")

## Feb 2016 Data

### Report 1

In [9]:
in_path_feb_2016_report = in_path
out_path_feb_2016_report = out_path

files = db.list_files(in_path_feb_2016_report)
files

['1_-_Incident(1).csv',
 '1_-_Incident(1).xlsx',
 '1_-_Incident(1)_metadata.csv',
 'CPD_Reporting_Party(1).csv',
 'CPD_Reporting_Party(1).xlsx',
 'CPD_Reporting_Party(1)_metadata.csv',
 'CPD_Witness(1).csv',
 'CPD_Witness(1).xlsx',
 'CPD_Witness(1)_metadata.csv',
 'crms_-_05j_complaint_and_investigator(1).csv',
 'crms_-_05j_complaint_and_investigator(1).xlsx',
 'crms_-_05j_complaint_and_investigator(1)_metadata.csv',
 'crms_-_05j_cpd_witness,_reporting_party,_victim(1).csv',
 'crms_-_05j_cpd_witness,_reporting_party,_victim(1).xlsx',
 'crms_-_05j_cpd_witness,_reporting_party,_victim(1)_metadata.csv',
 'crms_-_05j_Officer(1).csv',
 'crms_-_05j_Officer(1).xlsx',
 'crms_-_05j_Officer(1)_metadata.csv',
 'Incident_Address(1)_Block_Level.csv',
 'Incident_Address(1)_Block_Level.xlsx',
 'Incident_Address(1)_Block_Level_metadata.csv',
 'Involved_Member(1).csv',
 'Involved_Member(1).xlsx',
 'Involved_Member(1)_metadata.csv',
 'stars-for_involved_members(1).csv',
 'stars-for_involved_members(1).x

In [10]:
file = files[0]
df = db.download_file(in_path_feb_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/shootings-cpd-feb2016/1_-_incident(1).csv


['CRID',
 'Assignment',
 'Initial_Category',
 'Assigned_Team',
 'Team_Assigned_Date',
 'Investigator_Assigned',
 'Investigator_Assigned_Date',
 'Supervisor_Assigned',
 'IPRA_Closed_Date',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [11]:
df.head()

,CRID,Assignment,Initial_Category,Assigned_Team,Team_Assigned_Date,Investigator_Assigned,Investigator_Assigned_Date,Supervisor_Assigned,IPRA_Closed_Date,FOIA_Request_Number,Report_Produced_Date
0,1077428,IPRA,18A,GENERAL FIELD 9,2015-10-05 12:08:00,"ABBRUZZESE, WILLIAM",2015-10-07 00:00:00,"HUNT, JOSHUA",NaN,FOIA 15-6588,2016-01-12
1,1077328,IPRA,18A,GENERAL FIELD 9,2015-10-05 12:02:00,"WHITE, LAKENYA",2015-10-07 00:00:00,"HUNT, JOSHUA",NaN,FOIA 15-6588,2016-01-12
2,1077146,IPRA,18A,GENERAL FIELD 9,2015-09-15 13:23:00,"ABBRUZZESE, WILLIAM",2015-09-17 00:00:00,"HUNT, JOSHUA",NaN,FOIA 15-6588,2016-01-12
3,1076922,IPRA,18A,GENERAL FIELD 9,2015-08-31 16:41:00,"KILLEN, BRIAN",2015-09-03 00:00:00,"HUNT, JOSHUA",NaN,FOIA 15-6588,2016-01-12
4,1076815,IPRA,18A,GENERAL FIELD 9,2015-08-24 14:25:00,"LUKAS, JAMES",2015-08-27 00:00:00,"HUNT, JOSHUA",NaN,FOIA 15-6588,2016-01-12


In [12]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Investigator.Assignment',
 'Complaint.Initial.Category',
 'Investigator.Assigned_Team',
 'Investigator.Assigned_Team_Date',
 'Investigator.Full_Name',
 'Investigator.Appt_Date',
 'Investigator.Supervisor.Full_Name',
 'Complaint.Current.IPRA_Closed_Date',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [13]:
df.to_csv(out_path_feb_2016_report+file,index=False)

### Report 2

In [14]:
file = files[3]
df = db.download_file(in_path_feb_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/shootings-cpd-feb2016/cpd_reporting_party(1).csv


['CRID',
 'Officer_Reporting_Party',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [15]:
df.head()

,CRID,Officer_Reporting_Party,FOIA_Request_Number,Report_Produced_Date
0,1077428,"STAPLES, MELISSA",FOIA 15-6588,2016-01-12
1,1077328,"FLETCHER, CHRISTOPH",FOIA 15-6588,2016-01-12
2,1077146,"ALEXANDER, DANA",FOIA 15-6588,2016-01-12
3,1076922,"MC NAUGHTON, DAVID",FOIA 15-6588,2016-01-12
4,1076815,"WALLER, FRED",FOIA 15-6588,2016-01-12


In [16]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Officer.Reporting_Party.Full_Name',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [17]:
df.to_csv(out_path_feb_2016_report+file,index=False)

### Report 3

In [18]:
file = files[6]
df = db.download_file(in_path_feb_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/shootings-cpd-feb2016/cpd_witness(1).csv


['CRID', 'Officer_Witness', 'FOIA_Request_Number', 'Report_Produced_Date']

In [19]:
df.head()

,CRID,Officer_Witness,FOIA_Request_Number,Report_Produced_Date
0,1077428,"ANTICO, JOSEPH",FOIA 15-6588,2016-01-12
1,1077428,"GOZDAL III, MATTHEW",FOIA 15-6588,2016-01-12
2,1077428,"TIRADO, GLORIA",FOIA 15-6588,2016-01-12
3,1077328,"CHOATE, HEATH",FOIA 15-6588,2016-01-12
4,1077328,"CRONIN, ERIC",FOIA 15-6588,2016-01-12


In [20]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Officer.Witness.Full_Name',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [21]:
df.to_csv(out_path_feb_2016_report+file,index=False)

### Report 4

In [22]:
file = files[9]
df = db.download_file(in_path_feb_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/shootings-cpd-feb2016/crms_-_05j_complaint_and_investigator(1).csv


['CRID',
 'Beat',
 'Incident_Address',
 'City',
 'State',
 'Location_Code',
 'Location_Value',
 'Incident_Date',
 'Complaint_Date',
 'Investigator',
 'Assigned_Date',
 'Investigator_End_Date',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [23]:
df.head()

,CRID,Beat,Incident_Address,City,State,Location_Code,Location_Value,Incident_Date,Complaint_Date,Investigator,Assigned_Date,Investigator_End_Date,FOIA_Request_Number,Report_Produced_Date
0,1039179,2223.0,9100 S HALSTED ST,CHICAGO,IL,17,PUBLIC WAY - OTHER,2012-08-23 00:00:00,2012-08-23 00:00:00,"JONES, VINCENT",2012-04-18 16:15:00,2014-01-29 00:00:00,FOIA 15-6588,2015-12-30
1,1056520,432.0,3540 E 110TH ST,CHICAGO,IL,17,PUBLIC WAY - OTHER,2012-08-23 03:45:00,2012-08-23 00:00:00,"DAVIS, THERESA",2014-02-19 00:00:00,2014-05-23 00:00:00,FOIA 15-6588,2015-12-30
2,1059373,414.0,2005 E 75TH ST,CHICAGO,IL,17,PUBLIC WAY - OTHER,2013-01-07 13:46:00,2013-01-07 00:00:00,"WILSON, GRACE",2013-01-09 11:22:00,2015-06-16 00:00:00,FOIA 15-6588,2015-12-30
3,1071970,0.0,5259 S KING DR,CHICAGO,IL,17,PUBLIC WAY - OTHER,2014-10-12 00:35:00,2014-10-12 00:00:00,NaN,NaN,NaN,FOIA 15-6588,2015-12-30
4,1072125,815.0,4100 S PULASKI RD,CHICAGO,IL,17,PUBLIC WAY - OTHER,2014-10-20 21:47:00,2014-10-20 00:00:00,"DAVIS, THERESA",2014-11-12 00:00:00,NaN,FOIA 15-6588,2015-12-30


In [24]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Incident.Location.Beat',
 'Incident.Location.Address',
 'Incident.Location.City',
 'Incident.Location.State',
 'Incident.Location.Location_Code',
 'Incident.Location.Location_Value',
 'Incident.Time.Date',
 'Complaint.Date',
 'Investigator.Full_Name',
 'Investigator.Appt_Date',
 'Investigator.End_Date',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [25]:
df.to_csv(out_path_feb_2016_report+file,index=False)

### Report 5

In [26]:
file = files[12]
df = db.download_file(in_path_feb_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/shootings-cpd-feb2016/crms_-_05j_cpd_witness,_reporting_party,_victim(1).csv


['CRID',
 'Values',
 'Role',
 'CPD_or_Not',
 'Gender',
 'Ethnicity',
 'Year of Birth',
 'Star',
 'Position',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [27]:
df.head()

,CRID,Values,Role,CPD_or_Not,Gender,Ethnicity,Year of Birth,Star,Position,FOIA_Request_Number,Report_Produced_Date
0,1039179,1039179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FOIA 15-6588,2015-12-30
1,1039179,(Non-CPD witness or victim),VICTIM,NaN,F,BLK,1990.0,NaN,NaN,FOIA 15-6588,2015-12-30
2,1056520,1056520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FOIA 15-6588,2015-12-30
3,1056520,(Non-CPD witness or victim),VICTIM,NaN,M,WHI,1988.0,NaN,NaN,FOIA 15-6588,2015-12-30
4,1056520,"CHRISTENSEN, AMY",WITNESS,CPD Sworn,F,WHI,1971.0,16016.0,POLICE OFFICER,FOIA 15-6588,2015-12-30


In [28]:
df = df.dropna(subset=['Role','CPD_or_Not','Gender','Ethnicity','Year of Birth','Star','Position']) 
df_noncpd = df[df['Values']=='(Non-CPD witness or victim)']
df_cpd = df[df['Values']!='(Non-CPD witness or victim)']


In [29]:
df_cpd_witness = df_cpd[df_cpd['Role']=='WITNESS']
df_cpd_victim = df_cpd[df_cpd['Role']=='VICTIM']
df_noncpd_witness = df_noncpd[df_noncpd['Role']=='WITNESS']
df_noncpd_victim = df_noncpd[df_noncpd['Role']=='VICTIM']

In [30]:
def column_renamer(column):
    if column in ['Gender','Ethnicity','Year of Birth','Star','Position']:
        return 'Officer_Witness_'+column
    else:
        return column
    
df_cpd_witness.columns = [column_renamer(column) for column in df_cpd_witness.columns.tolist() ]
df_noncpd_witness.columns = [column_renamer(column) for column in df_cpd_witness.columns.tolist()]
df_cpd_victim.columns = [column_renamer(column) for column in df_cpd_witness.columns.tolist()]
df_noncpd_victim.columns = [column_renamer(column) for column in df_cpd_witness.columns.tolist()]

In [31]:
df_cpd_witness.columns.tolist()

['CRID',
 'Values',
 'Role',
 'CPD_or_Not',
 'Officer_Witness_Gender',
 'Officer_Witness_Ethnicity',
 'Officer_Witness_Year of Birth',
 'Officer_Witness_Star',
 'Officer_Witness_Position',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [32]:
df_cpd_witness.drop(['Values','Role','CPD_or_Not'], axis=1, inplace=True)
df_noncpd_witness.drop(['Values','Role','CPD_or_Not'], axis=1, inplace=True)
df_cpd_victim.drop(['Values','Role','CPD_or_Not'], axis=1, inplace=True)
df_noncpd_victim.drop(['Values','Role','CPD_or_Not'], axis=1, inplace=True)

df_cpd_witness.columns.tolist()

/Users/thudson/anaconda/envs/civis-model/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/thudson/anaconda/envs/civis-model/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


['CRID',
 'Officer_Witness_Gender',
 'Officer_Witness_Ethnicity',
 'Officer_Witness_Year of Birth',
 'Officer_Witness_Star',
 'Officer_Witness_Position',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [33]:
df_cpd_witness.columns = column_mapper(df_cpd_witness)
df_noncpd_witness.columns = column_mapper(df_noncpd_witness)
df_cpd_victim.columns = column_mapper(df_cpd_victim)
df_noncpd_victim.columns = column_mapper(df_noncpd_victim)

df_noncpd_victim.columns.tolist()

['CRID',
 'Officer.Witness.Gender',
 'Officer.Witness.Race_Code',
 'Officer.Witness.Birth_Year',
 'Officer.Witness.Star',
 'Officer.Witness.Position',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [34]:
df_cpd_witness.to_csv(out_path_feb_2016_report+'crms_-_05j'+'cpd_witness.csv',index=False)
df_noncpd_witness.to_csv(out_path_feb_2016_report+'crms_-_05j'+'non_cpd_witness.csv',index=False)
df_cpd_victim.to_csv(out_path_feb_2016_report+'crms_-_05j'+'cpd_victim.csv',index=False)
df_noncpd_victim.to_csv(out_path_feb_2016_report+'crms_-_05j'+'non_cpd_victim.csv',index=False)

### Other Files In Works

In [35]:
file = files[15]
df = db.download_file(in_path_feb_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/shootings-cpd-feb2016/crms_-_05j_officer(1).csv


['CRID',
 'Officer',
 'Star',
 'Gender',
 'Race',
 'Age',
 'Unit',
 'Position',
 'Appointment_Date',
 'Recommended_Finding',
 'Final_Finding',
 'Recommended_Action',
 'Final_Action',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [36]:
def column_renamer(column):
    if column in ['Gender','Race','Age','Star','Position','Unit','Appointment_Date']:
        return 'Officer_Accused_'+column
    else:
        return column
    
df.columns = [column_renamer(column) for column in df.columns.tolist() ]
df.columns.tolist()

['CRID',
 'Officer',
 'Officer_Accused_Star',
 'Officer_Accused_Gender',
 'Officer_Accused_Race',
 'Officer_Accused_Age',
 'Officer_Accused_Unit',
 'Officer_Accused_Position',
 'Officer_Accused_Appointment_Date',
 'Recommended_Finding',
 'Final_Finding',
 'Recommended_Action',
 'Final_Action',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [37]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Officer.Accused.Full_Name',
 'Officer.Accused.Star',
 'Officer.Accused.Gender',
 'Officer.Accused.Race_Code',
 'Officer.Accused.Birth_Year',
 'Officer.Accused.Unit',
 'Officer.Accused.Position',
 'Officer.Accused.Appointed_Date',
 'Complaint.Initial.Recommended_Finding',
 'Complaint.Final.Finding',
 'Complaint.Initial.Recommended_Discipline',
 'Complaint.Final.Recommended_Discipline',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [38]:
df.to_csv(out_path_feb_2016_report+file,index=False)

### Victim Detainee

In [39]:
file = files[18]
df = db.download_file(in_path_feb_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/shootings-cpd-feb2016/incident_address(1)_block_level.csv


['CRID',
 'Incident_Date',
 'Incident_Address',
 'Incident_Apt',
 'Incident_City',
 'Incident_State',
 'Incident_Zip',
 'District',
 'Incident_Start_Date',
 'Incident_End_Date',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [40]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Incident.Time.Date',
 'Incident.Location.Address',
 'Incident.Location.Apt_Nbr',
 'Incident.Location.City',
 'Incident.Location.State',
 'Incident.Location.Zip',
 'Incident.Location.District',
 'Incident.Time.Start_Time',
 'Incident.Time.End_Time',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [41]:
df.to_csv(out_path_feb_2016_report+file,index=False)

### Stars

In [42]:
file = files[21]
df = db.download_file(in_path_feb_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/shootings-cpd-feb2016/involved_member(1).csv


['CRID', 'Involved_Officer', 'FOIA_Request_Number', 'Report_Produced_Date']

In [43]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Officer.Involved.Full_Name',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [44]:
df.to_csv(out_path_feb_2016_report+file,index=False)

### Complaint & Investigator

In [45]:
file = files[24]
df = db.download_file(in_path_feb_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/shootings-cpd-feb2016/stars-for_involved_members(1).csv


['Officer_Last_Name',
 'Officer_First_Name',
 'Star',
 'Officer_Description',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [46]:
df.columns.values[0]='Involved_Officer_Last_Name'
df.columns.values[1]='Involved_Officer_First_Name'
df.columns.values[2]='Involved_Officer_Star'
df.columns.values[3]='Involved_Officer_Description'
df.columns = column_mapper(df)
df.columns.tolist()

['Officer.Involved.Last_Name',
 'Officer.Involved.First_Name',
 'Officer.Involved.Star',
 'Officer.Involved.Description',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [47]:
df.to_csv(out_path_feb_2016_report+file,index=False)

### Witness Reporting Party, and Victim

In [48]:
file = files[27]
df = db.download_file(in_path_feb_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/shootings-cpd-feb2016/victim-detainee_demographics(1).csv


['CRID',
 'Detainee_Age',
 'Detainee_Gender',
 'Detainee_Race',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [49]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Detainee.Age',
 'Detainee.Gender',
 'Detainee.Race_Code',
 'FOIA_Request_Number',
 'Report_Produced_Date']

In [50]:
df.to_csv(out_path_feb_2016_report+file,index=False)